In [ ]:
######################################################################
######################################################################
###### WRITTEN BY NEHA HUNKA, NHUNKA@UMD.EDU #########################
######################################################################
######################################################################

# This notebook shows how to integrate Sudan NFI-estimated AGBD and Volume, 
# Forest probability map and GEDI Forest Height estimates (see https://glad.umd.edu/dataset/gedi). 
# It relies on the computational power of the INLA Package in R (https://www.r-inla.org/).  

################################# ################################# #############################
################################# Geostatistical Model-based Estimation #########################
################################# ################################# #############################

# Our target variable (AGBD) at any given location (s) is assumed to follow the following linear model
# 
# \begin{align}
# y(s) = (\alpha + \tilde{\alpha}(s)) + (\beta + \tilde{\beta}(s)).x1(s) + (\eta + \tilde{\eta}(s)).x2(s) + \epsilon(s) 
# \end{align}
# 
# Most of the values should be familiar. $\alpha$, $\beta$ and $\eta$ are constant regression parameters. 
# x1 and x2 are our covariates (here, the FNF probability map and the GEDI-based height map). 
# $\alpha$, $\beta$ and $\eta$ have associated errors $\tilde{\alpha}$, $\tilde{\beta}$ and $\tilde{\eta}$ 
#   that are spatially autocorrelated. These errors capture the spatially-varying component of 
# $\alpha$, $\beta$ and $\eta$. In a Bayesian hierarchical framework, it is possible to estimate 
# the spatial covariance parameters and, hence, propagate their uncertainties through to the 
# prediction of the outcome variable (Babcock et al. 2015). 
# 
# A caveat, however, is that solving the above model is natively is computationally expensive when 
# the number of observations becomes large. We can circumvent this by splitting Sudan up into 
# (very) small regions. This is done by the use of a mesh, and approximating the model outputs 
# for each node/vertex of the mesh. First, define an underlying "mesh", which draws a graph of 
# triangles across the study area. On the vertices of triangles, we define a neighbor-based 
# spatial model where two vertices are neighbors if there is a triangle edge connecting them.
# To extend this to continuous locations, the value of $\tilde{\alpha}$, $\tilde{\beta}$ and 
# $\tilde{\eta}$ at any location (s) lying within a triangle is linearly interpolated from the 
# 3 vertices of the containing triangle. The reason why this is more computationally efficient 
# is that the process on the vertices is sparse, i.e. for any given vertex, it only needs the 
# information from the connected vertices. This will become clearer as we go through the code. 

############# AT THE START, WE INSTALL PACKAGES WE NEED ###############

# We use a package that might be new to some of you, called "INLA" 
# This package helps implement Bayesian methods and helps make some of our steps fast/easy to implement

############# OPEN R, THEN RUN THE FOLLOWING COMMAND ###################

install.packages("INLA",repos=c(getOption("repos"),INLA="https://inla.r-inla-download.org/R/stable"), dep=TRUE)
install.packages("fmesher", dependencies = TRUE)
install.packages("MatrixModels", type = "source")
install.packages("exactextractr")
install.packages("inlabru")
install.packages("sn" ,dependencies = TRUE)
packages <- c("terra","dplyr","spdep", "exactextractr", "sf","ggplot2","viridis","sn","fmesher","exactextractr","fields")
package.check <- lapply(packages, FUN = function(x) {
  if (!require(x, character.only = TRUE)) {
    install.packages(x, dependencies = TRUE)
    library(x, character.only = TRUE, quietly=TRUE)
  }
})
Sys.setenv("AWS_DEFAULT_REGION" = 'us-west-2')

######################################################
############## LOAD PACKAGES #########################
######################################################

library("fmesher")
library(MatrixModels)
library(Matrix)
library(INLA)
library(inlabru)
library("jpeg")
library(gstat)

library(sf)
library(terra)
library(dplyr)
library(spdep)
library(raster)
# library(exactextractr)

library(ggplot2)
library(viridis)
library(stringr)
# library(sn)

setwd("/home/sepal-user/Sudan_2025_FRL")

#################################################################
# ##### EXTRACT COVARIATE VALUES AT NFI PLOTS ###################
#################################################################

NFI <- st_read("Sudan_NFI_Volume.gpkg") %>% st_transform("epsg:6933")
DATA = NFI %>% st_drop_geometry()

GEDI_L4A <- rast("GEDI_L4A_AGB_Sudan_6933.tif")
GEDI_L4A.plot = exactextractr::exact_extract(GEDI_L4A,st_buffer(NFI,250), 'mean')
DATA$gedi.agbd <- GEDI_L4A.plot

FNF <- rast("Sudan_FNF_probability_2023_updated_6933.tif") 
FNF.plot = exactextractr::exact_extract(FNF,st_buffer(NFI,250), 'mean')
DATA$FNF.prob <- FNF.plot
# ##### write.csv(DATA,"GMB_Test/NFI_and_COVARIATES.csv",row.names=FALSE)

###############################################################################
# ##### CHECK WHAT DATA LOOKS LIKE BEFORE MODELING ############################
###############################################################################

options(repr.plot.width=5, repr.plot.height=5)
NFI = read.csv("DATA.csv")
plot(NFI$AG_Biomass..Mg.ha.,NFI$FNF)
plot(NFI$AG_Biomass..Mg.ha.,NFI$GEDI_L4A,xlim=c(0,70),ylim=c(0,70))

LM_model <- (lm(NFI$AG_Biomass..Mg.ha.~ NFI$GEDI_L4A))
summary(LM_model)
predictions <- predict(LM_model)
residuals <- NFI$AG_Biomass..Mg.ha. - predictions
rmse <- sqrt(mean(residuals^2))
print(paste("RMSE:", rmse))

ggplot(NFI, aes(x = AG_Biomass..Mg.ha., y = GEDI_L4A)) +
  geom_point() +  # Scatter points
  geom_smooth(method = "lm", se = FALSE, color = "red") +  # Fit line
  labs(title = "Scatter Plot with Fitted Line", x = "AG_Biomass..Mg.ha.", y = "GEDI_L4A") + theme_minimal()

formula = (AG_Biomass..Mg.ha.)~ (GEDI_L4A)
INLA_model <- inla(formula, data=NFI, family="gaussian",control.compute=list(config = T, dic = T, waic = T, cpo=TRUE)) #option cpo is a leave-one-out cross validation
summary(INLA_model)

#### Let's do a quick comparison, just to see how our outputs compare 
output <- cbind(summary(LM_model)$coef[,1:2],INLA_model$summary.fixed[,1:5]) # Looking great! 
colnames(output) <- c("lm():Mean","lm():SE","inla():Mean","inla():SD","inla():0.025quant","inla():0.5quant","inla():0.975quant")
output # Looking great! 

###############################################################################
####### ##############  DO MORAN'S I TEST ############## ######################
###############################################################################

Sudan <- st_read("Sudan_NE.gpkg")
Sudan <- Sudan %>% st_transform(crs=6933)
FNF <- rast("Sudan_FNF_probability_2023_updated_6933.tif")
THRESHOLD_FOREST_PROBABILITY = 10

DATA = read.csv("DATA.csv")
DATA$gedi.agbd <- (DATA$GEDI_L4A)
DATA$HEIGHT.agb <- (DATA$HEIGHT)
DATA$FNF.prob <- (DATA$FNF)

coordinates <- cbind(DATA$X, DATA$Y)
neighbors <- knn2nb(knearneigh(coordinates, k = 2)) 
listw <- nb2listw(neighbors)
moran_result <- moran.test(DATA$AG_Biomass..Mg.ha., listw)
print(moran_result)

# A very high standard deviation like 19.641 suggests that the observed value is significantly different from the expected mean (which is usually 0).
# This p-value is extremely small, indicating strong statistical significance.

coordinates(DATA) <- ~X + Y
v <- variogram(AG_Biomass..Mg.ha. ~ 1, DATA)
plot(v, main = "Semivariogram", xlab = "Lag Distance [m]", ylab = "Semivariance [(Mg/ha)^2]")

###################################################################
################ BEGIN MODELING - CREATE INLA MESH ################
###################################################################

options(repr.plot.width=8, repr.plot.height=6)
Sudan <- st_read("Sudan_NE.gpkg")
Sudan <- Sudan %>% st_transform(crs=6933)

DATA <- read.csv("DATA.csv")
DATA <- DATA[DATA$gedi.agbd > 0,]
DATA <- DATA[DATA$AG_Biomass..Mg.ha. > 0,]
factor <- (1/3)
DATA$AG_Biomass..Mg.ha. <- (DATA$AG_Biomass..Mg.ha.)^factor

loc.plot <- data.matrix(as.data.frame(cbind(DATA$X,DATA$Y)))
nfi.agbd <- DATA$AG_Biomass..Mg.ha. 
gedi.agbd <- DATA$gedi.agbd
FNF.prob <- DATA$FNF.prob

max.edge = 20*10^3
Sudan.buffer = st_buffer(Sudan, dist = max.edge*5) # We also make a 5 km buffer around Sudan to prevent boundary effects.
mesh = inla.mesh.2d(boundary = list(as(Sudan, "Spatial"), as(Sudan.buffer, "Spatial")), max.edge = c(max.edge, 3*max.edge), cutoff = 2*max.edge/3, offset = c(max.edge, 5*max.edge)) 
k = mesh$n ### This is the resulting number of vertices
loc.plot <- data.matrix(loc.plot) #In case loc.plot is a df rather than matrix, use "loc.plot <- data.matrix(loc.plot)"
plot(mesh)

n.plot = nrow(loc.plot)
A.plot = inla.spde.make.A(mesh = mesh, loc = loc.plot) # make a new projector matrix with the new mesh and the same plot locations. 

###############################################################
################ SET PRIORS AND RUN MODEL #####################
###############################################################

# Priors for the varying intercept alpha_tilde
spde.alpha = inla.spde2.pcmatern(mesh, 
                                 prior.range = c(30*10^3, 0.01), # This says the probability that the range is LESS than 30 km is 0.01
                                 prior.sigma = c(60, 0.01)) # This says the probability that the SD is GREATER than 60 Mg/ha is 0.01

# Priors for the varying coefficient beta_tilde
spde.beta = inla.spde2.pcmatern(mesh, 
                                prior.range = c(30*10^3, 0.01), # Same interpretations as above here
                                prior.sigma = c(1, 0.5)) # This says the probability that the SD is GREATER than 1 is 0.5


# Priors for the varying coefficient eta_tilde
spde.eta = inla.spde2.pcmatern(mesh, 
                               prior.range = c(30*10^3, 0.01), # Same interpretations as above here
                               prior.sigma = c(1, 0.5)) # This says the probability that the SD is GREATER than 1 is 0.5


### Construct our formula #####
formula = agbd ~
  -1 + # removes the automatic intercept so you can include your named intercept
  intercept + 
  L4A + 
  FNF + 
  f(alpha.spat, model = spde.alpha) + 
  f(beta.spat, model = spde.beta) + 
  f(eta.spat, model = spde.eta)

## Arrange our data so we can supply it to INLA to input in the formula above #####
stack = inla.stack(data = list(agbd = nfi.agbd),
                   A = list(1, # tell INLA we expect an intercept, and the projection matrix is just "1"
                            1, # tell INLA we expect a GEDI value, and the projection matrix for its parameter is just "1"
                            1, # tell INLA we expect a FNF value, and the projection matrix for its parameter is just "1"
                            A.plot, # tell INLA we have a mesh projected onto the plot locations for alpha-tilde 
                            Diagonal(x = gedi.agbd)%*%A.plot, # tell INLA to multiply our mesh with GEDI values for Beta-tilde
                            Diagonal(x = FNF.prob)%*%A.plot), # tell INLA to multiply our mesh with FNF values for eta-tilde
                   effects = list(
                     intercept = rep(1, n.plot), # The intercept just given an index equal to 1
                     L4A = gedi.agbd, # Supply INLA with the GEDI values
                     FNF = FNF.prob, # Supply INLA with the FNF values
                     alpha.spat = 1:k, # The random effects just need to given unique indices at each mesh node.
                     beta.spat = 1:k, # The random effects just need to given unique indices at each mesh node.
                     eta.spat = 1:k # The random effects just need to given unique indices at each mesh node.
                   ))

model_fit = inla(formula = formula, # Provide the formula
                 family = 'gaussian', # We assume our data follows a Gaussian generalized linear model (GLM)
                 data = inla.stack.data(stack), # These line and the one below are simply how we feed INLA the stack.
                 control.predictor = list(A = inla.stack.A(stack)),
                 control.compute = list(config = T, dic = T, waic = T, cpo = T),
                 control.inla = list(int.strategy = "eb"), # EB just fixes the hyperparameters at their maximum posterior (like maximum likelihood, but accounting for priors) values. The other methods manually integrate across the range of possible values.
                 verbose = TRUE)

summary(model_fit)
save(model_fit, file = "INLA_model_fit.RData")
hist(model_fit$cpo$pit, breaks = 20) 

PITs <- model_fit$cpo$pit
length(PITs[PITs >= 0.025 & PITs <= 0.975])/length(PITs)

###############################################################
######## SCATTER PLOT OF PREDICTIONS VS. NFI-ESTIMATES ########
###############################################################

samples = inla.posterior.sample(n = 250, result = model_fit) # Draw posterior samples from our model fit 

# Our model function
pred_fun = function(...){
  drop(intercept + 
         gedi.agbd*L4A +
         FNF.prob*FNF +
         A.plot%*%alpha.spat[1:k] +   
         Diagonal(x = gedi.agbd)%*%A.plot%*%beta.spat[1:k] + 
         Diagonal(x = FNF.prob)%*%A.plot%*%eta.spat[1:k]) +
    rnorm(nrow(A.plot), sd = sqrt(1/theta[1])) 
}

# Generate prediction samples, i.e. ask INLA to use the prediction function and samples to make predictions. 
pred.samples = (inla.posterior.sample.eval(fun = pred_fun,samples = samples)) #

# Model mean AGBD expectations and SD's at the grid locations
pred.mu = Matrix::rowMeans(pred.samples^(1/factor),na.rm=TRUE)
pred.sd = apply(pred.samples^(1/factor), 1, sd)
options(repr.plot.width=5, repr.plot.height=5)

OUTPUT <- do.call(rbind, Map(data.frame, PRED=pred.mu, NFI=nfi.agbd^(1/factor), PRED.SD=pred.sd, RES=nfi.agbd^(1/factor)-pred.mu))
options(repr.plot.width=6, repr.plot.height=6)
ggplot(data = OUTPUT, aes(x = NFI,y = PRED))+ geom_point()  + geom_abline(size=0.5,linetype=2,col="red",lwd=1.5) + theme_bw() + xlim(0,60) + ylim(0,60) + xlab("NFI AGBD") + ylab("GMB model predictions ") + ggtitle("NFI AGBD vs. square of GMB predictions at 10% testing locations") + theme(plot.title = element_text(color="red", size=12, face="bold.italic"),axis.text=element_text(size=20))

####################################################################
######################## HEURISTICS: R2 AND RMSE ###################
####################################################################

R2_backtrans = 1 - (sum((pred.mu - nfi.agbd^(1/factor))^2,na.rm=TRUE)/sum((nfi.agbd^(1/factor) - mean(nfi.agbd^(1/factor),na.rm=TRUE))^2,na.rm=TRUE))
R2_backtrans
RMSE = sqrt(mean((pred.mu - nfi.agbd^(1/factor))^2,na.rm=TRUE))
RMSE
BIAS = mean(pred.mu,na.rm=TRUE) - mean(nfi.agbd^(1/factor),na.rm=TRUE)
BIAS # Mg/ha

###########################################################################
############## RATE OF OVERLAP OF CREDIBILITY INTERVALS ###################
###########################################################################

Q25 <- apply(pred.samples^(1/factor), 1, quantile, probs = c(0.025), na.rm=TRUE)
Q975 <- apply(pred.samples^(1/factor), 1, quantile, probs = c(0.975), na.rm=TRUE)
sum((nfi.agbd^(1/factor) >= Q25) & (nfi.agbd^(1/factor) <= Q975), na.rm=TRUE)/length(nfi.agbd^(1/factor))

Q25 <- apply(pred.samples^(1/factor), 1, quantile, probs = c(0.025),  na.rm = TRUE)
Q975 <- apply(pred.samples^(1/factor), 1, quantile, probs = c(0.975),  na.rm = TRUE)
AGBD <- nfi.agbd^(1/factor)
Q25 <- Q25[AGBD > 10]
Q975 <- Q975[AGBD > 10]
AGBD <- AGBD[AGBD > 10]
sum((AGBD >= Q25) & (AGBD <= Q975), na.rm=TRUE)/length(AGBD)

Q25 <- apply(pred.samples^(1/factor), 1, quantile, probs = c(0.025),  na.rm = TRUE)
Q975 <- apply(pred.samples^(1/factor), 1, quantile, probs = c(0.975),  na.rm = TRUE)
AGBD <- nfi.agbd^(1/factor)
Q25 <- Q25[AGBD <= 10]
Q975 <- Q975[AGBD <= 10]
AGBD <- AGBD[AGBD <= 10]
sum((AGBD >= Q25) & (AGBD <= Q975), na.rm=TRUE)/length(AGBD)

######################################################################
####### PREDICTIONS AT SITES OF DEGRADATION REDONE ###################
######################################################################
THRESHOLD_FOREST_PROBABILITY = 0

load("INLA_model_fit.RData")
GEDI_L4A = rast("GEDI_L4A_AGB_Sudan_6933.tif")
FNF <- rast("Sudan_FNF_probability_2023_updated_6933.tif")

DEG <- read.csv("NFI_grid_sudan_CE_assessment.csv")
grid_to_predict <- st_as_sf(DEG, coords = c("location_x", "location_y"), crs = 4326) %>% st_transform("epsg:6933")
grid = as.data.frame(grid_to_predict %>% st_coordinates())

GEDI.pred = exactextractr::exact_extract(GEDI_L4A,st_buffer(grid_to_predict,21), 'mean')
grid$GEDI.pred <- GEDI.pred

FNF.pred = exactextractr::exact_extract(FNF,st_buffer(grid_to_predict,21), 'mean')
FNF.array <- FNF.pred
FNF.array[FNF.array<=THRESHOLD_FOREST_PROBABILITY] = 1 #NA (we want to prepare ALL degrdation sites, irrespective of FNF)
FNF.array[FNF.array>THRESHOLD_FOREST_PROBABILITY] = 1
grid$FNF.array = FNF.array

grid_original <- grid
grid <- grid[(!is.na(grid$FNF.array) & grid$GEDI.pred>0),]
DEG <- DEG[(!is.na(grid_original$FNF.array) & grid_original$GEDI.pred>0),]
grid_to_predict <- grid_to_predict[(!is.na(grid_original$FNF.array) & grid_original$GEDI.pred>0),]
loc.plot_T <- data.matrix(as.data.frame(cbind(grid$X,grid$Y)))

FNF.pred <- FNF.pred[(!is.na(FNF.array) & GEDI.pred>0)]
GEDI.pred <- GEDI.pred[(!is.na(FNF.array) & GEDI.pred>0)]

samples = inla.posterior.sample(n = 1000, result = model_fit) # Draw posterior samples from our model fit 
A.pred = inla.spde.make.A(mesh = mesh, loc = loc.plot_T) # Make a new projector matrix with the same mesh, but new plot locations 

# Our model function
pred_fun = function(...){
  drop(intercept + 
         GEDI.pred*L4A +
         FNF.pred*FNF +
         A.pred%*%alpha.spat[1:k] +   
         Diagonal(x = GEDI.pred)%*%A.pred%*%beta.spat[1:k] + 
         Diagonal(x = FNF.pred)%*%A.pred%*%eta.spat[1:k]) +
    rnorm(nrow(A.pred), sd = sqrt(1/theta[1])) 
}

# Generate prediction samples, i.e. ask INLA to use the prediction function and samples to make predictions at our testing plot locations. 
pred.samples = (inla.posterior.sample.eval(fun = pred_fun,samples = samples)) #
pred.samples[pred.samples < 0] = 0

# Model mean AGBD expectations and SD's at the grid locations
pred.mu = Matrix::rowMeans(pred.samples^(1/factor),na.rm=TRUE)
pred.sd = apply(pred.samples^(1/factor), 1, sd)

DEG['AGBD'] <- pred.mu
DEG['AGBD_SE'] <- pred.sd
write.csv(DEG,"NFI_grid_sudan_CE_assessment_AGBD.csv",row.names=FALSE) #Save the data

ggplot() + coord_sf(crs = 6933) + 
  geom_point(aes(x = loc.plot_T[,1], y = loc.plot_T[,2], col = pred.mu),size=0.5) +  # make a map to see plot locations and the AGBD values at each location
  scale_color_viridis(limits = c(0, 50)) + xlab("X location of testing plots") + ylab("Y location of testing plots") + ggtitle("Mean predicted AGBD") + theme(plot.title = element_text(color="red", size=14, face="bold.italic"))

ggplot() + coord_sf(crs = 6933) + 
  geom_point(aes(x = loc.plot_T[,1], y = loc.plot_T[,2], col = pred.sd),size=0.5) +  # make a map to see plot locations and the AGBD values at each location
  scale_color_viridis(limits = c(0, 50)) + xlab("X location of testing plots") + ylab("Y location of testing plots") + ggtitle("SD of predicted AGBD") + theme(plot.title = element_text(color="red", size=14, face="bold.italic"))

In [ ]:
pip install dgown

python

import os
import gdown

DIR = "/home/sepal-user/Sudan_2025_FRL"
if not os.path.exists(DIR): 
    os.mkdir(DIR) 
os.chdir(DIR)

urls = [
    'https://drive.google.com/file/d/1KFx3Is07yhDIfUTY8hI0VvmD9Xuzzshw/view?usp=drive_link',
    'https://drive.google.com/file/d/1c_S5G5r9KIZ1V4mA97bMx8QQ7now4rUE/view?usp=drive_link'
]

output_files = [
    'GEDI_L4A_AGB_Sudan_6933.tif',
    'Sudan_FNF_probability_2023_updated_6933.tif'
]

for url, output_file in zip(urls, output_files):
    print(f"Downloading {output_file} from {url}")
    gdown.download(url, output=output_file, fuzzy=True)